In [9]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer


In [10]:
from transformers import Trainer


In [11]:
import torch



In [12]:
import pandas as pd


In [13]:
from datasets import Dataset


In [14]:
import torch
!pip install SpeechRecognition
import speech_recognition as sr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 3.2 MB/s eta 0:00:0000:0100:01


In [15]:
!pip install gTTS

from gtts import gTTS

In [16]:
!pip install youtube-transcript-api
import os
from youtube_transcript_api import YouTubeTranscriptApi # This line should work after installation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 2.8 MB/s eta 0:00:00a 0:00:01


In [17]:
# Define the file_path variable before using it
file_path = "/content/week2.json"  # Replace your_file.json with the actual path to your file

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)


FileNotFoundError: [Errno 2] No such file or directory: '/content/week2.json'

In [ ]:
 import pandas as pd

def create_dataset(data):  # Assuming you want to create a function to process the data
    contexts = []
    questions = []
    answers = []

    for item in data['data']:
        for paragraph in item['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = qa['answers'][0]  # Assuming one answer per question
                contexts.append(context)
                questions.append(question)
                answers.append({"text": [answer['text']], "answer_start": [answer['answer_start']]})
    # Indentation corrected here
    df = pd.DataFrame({"context": contexts, "question": questions, "answers": answers})

    # Assumed you want to import and use Dataset (likely from datasets library)
    from datasets import Dataset
    return Dataset.from_pandas(df)

In [ ]:
import json # Import the json module to access json.load()

file_path = r"/content/week2.json"  # Replace with your JSON file path

# Use json.load() to read the JSON data from the file
with open(file_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

In [ ]:
model_name = "bert-base-uncased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
    return tokenizer(
        examples["question"],
        examples["context"],
        truncation=True,
        padding="max_length",
        max_length=512
    )



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import json
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

# Load data from JSON file
file_path = r"/content/week2.json"  # Replace with your JSON file path
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Create Dataset object using the function you already defined
dataset = create_dataset(data)

# ... (Rest of your code)

# Load tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocess function
def preprocess(examples):
    return tokenizer(
        examples["question"],
        examples["context"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

# Apply the preprocessing to the Dataset
tokenized_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)



Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
!pip install -q datasets transformers scikit-learn

import numpy as np
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments, AutoModelForQuestionAnswering, AutoTokenizer

# ... (Your previous code to load and preprocess the dataset) ...

# Convert the Hugging Face Dataset to NumPy arrays for the specific columns
# Assuming you want to convert the 'input_ids', 'attention_mask', and 'token_type_ids' columns:
dataset_np = {
    'input_ids': np.array(tokenized_dataset['input_ids']),
    'attention_mask': np.array(tokenized_dataset['attention_mask']),
    'token_type_ids': np.array(tokenized_dataset['token_type_ids'])  # Add if this column exists
}

# Split the NumPy arrays into train and evaluation sets
train_indices, eval_indices = train_test_split(np.arange(len(dataset_np['input_ids'])), test_size=0.2, random_state=42)  # Adjust test_size as needed

# Create Hugging Face Datasets from the split indices 
train_dataset = tokenized_dataset.select(train_indices)
eval_dataset = tokenized_dataset.select(eval_indices)

# ... (Rest of your training code) ...

In [ ]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")



('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [ ]:
def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening for your question...")
        audio = recognizer.listen(source)
    try:
        question = recognizer.recognize_google(audio)
        print(f"You asked: {question}")
        return question
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None

def text_to_speech(text):
    tts = gTTS(text)
    tts.save("response.mp3")
    os.system("start response.mp3")  # Use 'start' for Windows; 'open' for MacOS

In [ ]:
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
    )
    return answer


In [ ]:
def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        transcript_text = " ".join([entry['text'] for entry in transcript])
        return transcript_text
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None


In [ ]:
!sudo apt-get update
!sudo apt-get install portaudio19-dev
!pip install PyAudio

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
!pip install SpeechRecognition

In [ ]:
!pip install --upgrade pip  # Upgrade pip
!pip cache purge           # Clear pip cache
!pip install SpeechRecognition --index-url https://pypi.org/simple/ # Install with specific index

Files removed: 2
Looking in indexes: https://pypi.org/simple/


In [ ]:
!sudo apt-get update
!sudo apt-get install alsa-utils

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
import speech_recognition as sr
import pyaudio

def speech_to_text():
    recognizer = sr.Recognizer()

    # List available audio devices
    print("Available audio devices:")
    p = pyaudio.PyAudio()
    for i in range(p.get_device_count()):
        device_info = p.get_device_info_by_index(i)
        print(f"{i}: {device_info['name']}")
    p.terminate()

    # Allow user to select audio device
    device_index = int(input("Select audio device index: "))

    with sr.Microphone(device_index=device_index) as source:
        print("Listening for your question...")
        audio = recognizer.listen(source)
    try:
        question = recognizer.recognize_google(audio)
        print(f"You asked: {question}")
        return question
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None

In [ ]:
!arecord -l  # List available audio input devices
!aplay -l   # List available audio output devices
# If you don't see any input devices listed, you may need to install audio drivers or enable audio in system settings

arecord: device_list:274: no soundcards found...
aplay: device_list:274: no soundcards found...


In [ ]:
!pip install speech_recognition PyAudio
import speech_recognition as sr
import pyaudio  # Import pyaudio explicitly

def speech_to_text():
    recognizer = sr.Recognizer()

    # Print available audio devices
    print("Available audio devices:")
    p = pyaudio.PyAudio()
    for i in range(p.get_device_count()):
        device_info = p.get_device_info_by_index(i)
        print(f"{i}: {device_info['name']}")
    p.terminate()

    with sr.Microphone() as source: # Initialize source, after printing audio devices
        print("Listening for your question...")
        audio = recognizer.listen(source)
    try:
        question = recognizer.recognize_google(audio)
        print(f"You asked: {question}")
        return question
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
        return None

ERROR: Could not find a version that satisfies the requirement speech_recognition (from versions: none)
ERROR: No matching distribution found for speech_recognition


In [ ]:
import pyaudio

p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')

for i in range(0, numdevices):
    if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
        print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

In [ ]:
if __name__ == "__main__": # Changed _name_ to __name__
    video_id = "w8Dq8blTmSA"  # Replace this with the actual YouTube video ID of NPTEL
    context = get_transcript(video_id)  # Fetch the transcript from the NPTEL video

    if context:
        print("Transcript fetched successfully.")
        question = speech_to_text()  # Use the STT to capture the question

        if question:
            answer = answer_question(question, context)
            print(f"Answer: {answer}")
            text_to_speech(answer)  # Convert the answer to speech
        else:
            print("No question detected.")
    else:
        print("Could not fetch transcript.")

Transcript fetched successfully.
Available audio devices:


OSError: No Default Input Device Available